<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%206%20-%20Aprendizaje%20por%20Transferencia%20y%20puesta%20a%20punto%20de%20parámetros%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Paso 1: Instalación de dependencias y configuración del entorno en GPU

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [ ]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2022-10-15 16:05:35--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 172.217.15.80, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   166MB/s    in 0.4s    

2022-10-15 16:05:36 (166 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [2]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.9.2'

### Descomprimir el dataset de Perros vs Gatos

In [3]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [4]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [5]:
zip_object.extractall("./")

In [6]:
zip_object.close()

### Configurar las rutas al dataset

In [7]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [8]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [9]:
IMG_SHAPE = (128, 128, 3)

In [10]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

### Congelar el modelo base

In [12]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [13]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [15]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [16]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [17]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

### Compilar el modelo

In [19]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [20]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [21]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [22]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [23]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/8
16/16 [==============================] - 19s 581ms/step - loss: 0.9564 - accuracy: 0.4415 - val_loss: 0.8755 - val_accuracy: 0.4650
Epoch 2/8
16/16 [==============================] - 8s 490ms/step - loss: 0.7926 - accuracy: 0.5050 - val_loss: 0.7546 - val_accuracy: 0.5390
Epoch 3/8
16/16 [==============================] - 8s 487ms/step - loss: 0.6860 - accuracy: 0.6005 - val_loss: 0.6569 - val_accuracy: 0.6340
Epoch 4/8
16/16 [==============================] - 8s 490ms/step - loss: 0.5971 - accuracy: 0.6825 - val_loss: 0.5755 - val_accuracy: 0.7070
Epoch 5/8
16/16 [==============================] - 8s 494ms/step - loss: 0.5239 - accuracy: 0.7525 - val_loss: 0.5065 - val_accuracy: 0.7770
Epoch 6/8
16/16 [==============================] - 8s 488ms/step - loss: 0.4610 - accuracy: 0.8090 - val_loss: 0.4480 - val_accuracy: 0.8190
Epoch 7/8
16/16 [==============================] - 8s 483ms/step - loss: 0.4089 - accuracy: 0.8460 - val_loss: 0.3976 - val_accuracy: 0.8560
Epoch 8/8
16

### Evaluar el modelo de aprendizaje por transferencia

In [26]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 3s 318ms/step - loss: 0.3548 - accuracy: 0.8690


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8690000176429749


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [28]:
base_model.trainable = True

In [36]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [30]:
fine_tune_at = 100

In [31]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Puesta a punto

In [33]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/5
16/16 [==============================] - 15s 575ms/step - loss: 0.1869 - accuracy: 0.9235 - val_loss: 0.0967 - val_accuracy: 0.9610
Epoch 2/5
16/16 [==============================] - 8s 503ms/step - loss: 0.0251 - accuracy: 0.9980 - val_loss: 0.1502 - val_accuracy: 0.9550
Epoch 3/5
16/16 [==============================] - 8s 498ms/step - loss: 0.0118 - accuracy: 0.9995 - val_loss: 0.1351 - val_accuracy: 0.9580
Epoch 4/5
16/16 [==============================] - 8s 497ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.1340 - val_accuracy: 0.9610
Epoch 5/5
16/16 [==============================] - 8s 502ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1342 - val_accuracy: 0.9620


### Evaluar el modelo re calibrado

In [34]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [35]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9620000123977661
